In [1]:
import gensim
from gensim.models import KeyedVectors
import numpy as np
import torch
from torch import nn, autograd, optim
import torch.nn.functional as F
import time
from tqdm import tqdm
import math

In [3]:
fvec = KeyedVectors.load_word2vec_format('vec_100d.txt', binary=False)
word_vec = fvec.vectors
vocab = ['<PAD>', '<BOS>', '<EOS>', '<UNK>']
vocab.extend(list(fvec.vocab.keys()))
word_vec = np.concatenate((np.array([[0]*word_vec.shape[1]] * 4), word_vec))
word_vec = torch.tensor(word_vec)

In [4]:
word_to_idx = {ch: i for i, ch in enumerate(vocab)}
idx_to_word = {i: ch for i, ch in enumerate(vocab)}

In [6]:
essays = []
topics = []
with open('composition.txt', 'r') as f:
    for line in f:
        essay, topic = line.replace('\n', '').split(' </d> ')
        words = essay.split(' ')
        new_words = []
        for word in words:
            if word in word_to_idx:
                new_words.append(word)
            else:
                for i in range(len(word)):
                    new_words.append(word[i])
        essays.append(new_words)
        topics.append(topic.split(' '))

In [8]:
corpus_indice = list(map(lambda x: [word_to_idx[w] for w in x], essays[:8000]))
topics_indice = list(map(lambda x: [word_to_idx[w] for w in x], topics[:8000]))

In [9]:
length = list(map(lambda x: len(x), corpus_indice))

In [10]:
def tav_data_iterator(corpus_indice, topics_indice, batch_size, num_steps):
    epoch_size = len(corpus_indice) // batch_size
    for i in range(epoch_size):
        raw_data = corpus_indice[i*batch_size: (i+1)*batch_size]
        key_words = topics_indice[i*batch_size: (i+1)*batch_size]
        data = np.zeros((len(raw_data), num_steps+1), dtype=np.int64)
        for i in range(batch_size):
            doc = raw_data[i]
            tmp = [1]
            tmp.extend(doc)
            tmp.extend([2])
            tmp = np.array(tmp, dtype=np.int64)
            _size = tmp.shape[0]
            data[i][:_size] = tmp
        key_words = np.array(key_words, dtype=np.int64)
        x = data[:, 0:num_steps]
        y = data[:, 1:]
        mask = np.float32(x != 0)
        x = torch.tensor(x)
        y = torch.tensor(y)
        mask = torch.tensor(mask)
        key_words = torch.tensor(key_words)
        yield(x, y, mask, key_words)

In [11]:
class TAVLSTM(nn.Module):
    def __init__(self, hidden_dim, embed_dim, num_layers, weight,
                 num_labels, bidirectional, dropout=0.5, **kwargs):
        super(TAVLSTM, self).__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.num_labels = num_labels
        self.bidirectional = bidirectional
        if num_layers <= 1:
            self.dropout = 0
        else:
            self.dropout = dropout
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.rnn = nn.GRU(input_size=self.embed_dim, hidden_size=self.hidden_dim,
                          num_layers=self.num_layers, bidirectional=self.bidirectional,
                          dropout=self.dropout)
        if self.bidirectional:
            self.decoder = nn.Linear(hidden_dim * 2, self.num_labels)
        else:
            self.decoder = nn.Linear(hidden_dim, self.num_labels)
        
    def forward(self, inputs, topics, hidden=None):
        embeddings = self.embedding(inputs)
        topics_embed = self.embedding(topics)
        topics_embed = topics_embed.mean(dim=1)
        for i in range(embeddings.shape[0]):
            embeddings[i][0] = topics_embed[i]
        states, hidden = self.rnn(embeddings.permute([1, 0, 2]).float(), hidden)
        outputs = self.decoder(states.reshape((-1, states.shape[-1])))
        return(outputs, hidden)
    
    def init_hidden(self, num_layers, batch_size, hidden_dim, **kwargs):
        hidden = torch.zeros(num_layers, batch_size, hidden_dim)
        return hidden

In [12]:
def predict_rnn(topics, num_chars, model, device, idx_to_word, word_to_idx):
    output = [1]
    topics = [word_to_idx[x] for x in topics]
    topics = torch.tensor(topics)
    hidden = torch.zeros(num_layers, 1, hidden_dim)
    if use_gpu:
        hidden = hidden.to(device)
        topics = topics.to(device)
#         hidden = hidden.cuda()
#         topics = topics.cuda()
    for t in range(num_chars):
        X = torch.tensor(output).reshape((1, len(output)))
        if use_gpu:
            X = X.to(device)
#             X = X.cuda()
        pred, hidden = model(X, topics, hidden)
        if pred.argmax(dim=1)[-1] == 2:
            break
        else:
            output.append(int(pred.argmax(dim=1)[-1]))
    return(''.join([idx_to_word[i] for i in output[1:]]))

In [13]:
embedding_dim = 300
hidden_dim = 256
lr = 1e2
momentum = 0.0
num_epoch = 50
use_gpu = True
num_layers = 1
bidirectional = False
batch_size = 1
device = torch.device('cuda:0')
loss_function = nn.CrossEntropyLoss()

In [14]:
model = TAVLSTM(hidden_dim=hidden_dim, embed_dim=embedding_dim, num_layers=num_layers,
                num_labels=len(vocab), weight=word_vec, bidirectional=bidirectional)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
if use_gpu:
#     model = nn.DataParallel(model)
    model.to(device)

In [15]:
predict_rnn(['变形金刚', '三星级'], 5, model, device, idx_to_word, word_to_idx)

'钝化妖怪韩梓轩梦还在梦还在'

In [16]:
since = time.time()
for epoch in range(num_epoch):
    start = time.time()
    num, total_loss = 0, 0
#     if epoch == 5000:
#         optimizer.param_groups[0]['lr'] = lr * 0.1
    data = tav_data_iterator(corpus_indice, topics_indice, batch_size, max(length)+1)
    hidden = model.init_hidden(num_layers, batch_size, hidden_dim)
    weight = torch.ones(len(vocab))
    weight[0] = 0
    for X, Y, mask, topics in tqdm(data):
        num += 1
        hidden.detach_()
        if use_gpu:
            X = X.to(device)
            Y = Y.to(device)
            mask = mask.to(device)
            topics = topics.to(device)
            hidden = hidden.to(device)
            weight = weight.to(device)
#             X = X.cuda()
#             Y = Y.cuda()
#             mask = mask.cuda()
#             topics = topics.cuda()
#             hidden = hidden.cuda()
        optimizer.zero_grad()
        output, hidden = model(X, topics, hidden)
        l = F.cross_entropy(output, Y.t().reshape((-1,)), weight)
        l.backward()
        norm = nn.utils.clip_grad_norm_(model.parameters(), 1e-2)
        optimizer.step()
        total_loss += l.item()
    end = time.time()
    s = end - since
    h = math.floor(s / 3600)
    m = s - h * 3600
    m = math.floor(m / 60)
    s -= m * 60
    if(epoch % 10 == 0) or (epoch == (num_epoch - 1)):
        print('epoch %d/%d, loss %.4f, norm %.4f, time %.3fs, since %dh %dm %ds'
              %(epoch+1, num_epoch, total_loss / num, norm, end-start, h, m, s))
        print(predict_rnn(['变形金刚', '三星级'], 100, model, device, idx_to_word, word_to_idx))

47it [00:01, 24.18it/s]


epoch 1/50, loss 10.2006, norm 1.5977, time 1.962s, since 0h 0m 1s


3it [00:00, 25.28it/s]

，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，，


47it [00:01, 24.76it/s]
47it [00:01, 24.88it/s]
47it [00:01, 24.93it/s]
47it [00:01, 24.95it/s]
47it [00:01, 24.92it/s]
47it [00:01, 24.87it/s]
47it [00:01, 24.88it/s]
47it [00:01, 24.88it/s]
47it [00:01, 24.80it/s]
47it [00:01, 24.82it/s]


...

epoch 31/50, loss 0.4017, norm 0.7018, time 1.918s, since 0h 1m 0s
不知道为什么我一看见机器人打架咯吱咯吱的就觉得各种治愈……


47it [00:01, 24.28it/s]
47it [00:01, 24.34it/s]
47it [00:01, 24.39it/s]
47it [00:01, 24.06it/s]
47it [00:01, 24.17it/s]
47it [00:01, 24.09it/s]
47it [00:01, 24.18it/s]
47it [00:01, 24.27it/s]
47it [00:01, 24.25it/s]
47it [00:01, 24.15it/s]
3it [00:00, 25.07it/s]

epoch 41/50, loss 0.2562, norm 0.2438, time 1.949s, since 0h 1m 20s
不知道为什么我一看见机器人打架咯吱咯吱的就觉得各种治愈……


47it [00:01, 24.03it/s]
47it [00:01, 24.16it/s]
47it [00:01, 24.24it/s]
47it [00:01, 24.27it/s]
47it [00:01, 24.36it/s]
47it [00:01, 24.20it/s]
47it [00:01, 24.13it/s]
47it [00:01, 24.36it/s]
47it [00:01, 23.95it/s]

epoch 50/50, loss 0.2541, norm 0.2653, time 1.964s, since 0h 1m 37s
不知道为什么我一看见机器人打架咯吱咯吱的就觉得各种治愈……
